In [1]:
import keras.backend as K
## one hot encoded labels
import numpy as np
from keras.utils import to_categorical

X_train = np.load('X_train_Fer2013.npy')
X_test = np.load('X_test_Fer2013.npy')
y_train = np.load('y_train_Fer2013.npy')
y_test = np.load('y_test_Fer2013.npy')
arousal_index = np.load('pseudo_arousal.npy')

import keras.backend as K
## one hot encoded labels
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

### Ready for CNN shape change

img_rows, img_cols = 48, 48

if K.image_data_format() == 'channels_first':
    X_train= X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### Feeding Generated Neutral Face info.

X_train_Neutral = np.load('evae_conditional_fer2013_sample_neutral_sample_neutral_a=0_v1.npy')


In [3]:
print X_train_Neutral.shape
print X_train.shape

(32298, 48, 48, 1)
(32298, 48, 48, 1)


In [4]:
### Making Binary Classification Training dataset 
### 1:x_train;   0: x_train_neutral 

X_train_binary = np.concatenate((X_train[:16149] ,X_train_Neutral[:16149]), axis = 0) ## shape[32298*2, 48, 48, 1]
y_train_binary_1 = np.ones(X_train.shape[0])[:16149]
y_train_binary_0 = np.zeros(X_train.shape[0])[:16149]
y_train_binary  = np.concatenate((y_train_binary_1, y_train_binary_0), axis = 0)


In [5]:
print X_train_binary.shape
print y_train_binary.shape
print y_train_binary[16148:16150] ## checking if half is 1; and the second half is 0


(32298, 48, 48, 1)
(32298,)
[1. 0.]


##### Tryout for making binary classification via a simple CNN net 

1. 

In [6]:
32298/2

16149

In [7]:
from keras.layers import Conv2D, BatchNormalization, AveragePooling2D, PReLU, Dropout, Dense
from keras.layers import Flatten, Activation,Subtract
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Concatenate
from keras import Model
from keras.layers import Dot,Multiply,Subtract,Add,BatchNormalization, Average, Maximum
from keras.layers import Lambda, Input, Dense
from keras import backend as K
from keras.layers.advanced_activations import PReLU

input_shape = (48,48,1)

num_classes = 7


### Network
x_binary = Input(shape=input_shape)
#arousal = Input(shape=(1,), name = 'fake_arousal_index')
###  Tryout more variations of initlisation techniques 
### without any pre-training 
### without any weight initilaistion 

##
h1 = Conv2D(16, (7, 7), padding='same',
                        input_shape=input_shape, name = 'block1_conv1')(x_binary)
h1 = PReLU()(h1)
h1 = BatchNormalization()(h1)
h1 = AveragePooling2D(pool_size=(5, 5),strides=(2, 2), padding='same')(h1)
h1 = Dropout(.5)(h1)

##
h2 = Conv2D(32, (5, 5), padding='same',
                        input_shape=input_shape, name = 'block1_conv2')(h1)
h2 = PReLU()(h2)
h2 = BatchNormalization()(h2)
h2 = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h2)
h2 = Dropout(.5)(h2)

##
h3 = Conv2D(32, (3, 3), padding='same',
                        input_shape=input_shape, name = 'block1_conv3')(h2)
h3 = PReLU()(h3)
h3 = BatchNormalization()(h3)
h3 = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h3)
h3 = Dropout(.5)(h3)


h3 = Flatten()(h3)
h4 = Dense(512)(h3)
h4 = PReLU()(h4)
h5 = Dropout(0.5)(h4)
h5 = BatchNormalization()(h5)
h_class = Dense(50, activation = 'relu')(h5)
prediction_binary = Dense(1, activation='sigmoid', name = 'output_binary')(h_class)
label_binary = Input((1,))
#y_true = Input(shape= (7,), name = 'y_true')

In [8]:
### Network 2 for standard multi class

input_shape = (48,48,1)

num_classes = 7



x_multi = Input(shape=input_shape)
#arousal = Input(shape=(1,), name = 'fake_arousal_index')
###  Tryout more variations of initlisation techniques 
### without any pre-training 
### without any weight initilaistion 

##
h1_ = Conv2D(16, (7, 7), padding='same',
                        input_shape=input_shape,)(x_multi)
h1_ = PReLU()(h1_)
h1_ = BatchNormalization()(h1_)
h1_ = AveragePooling2D(pool_size=(5, 5),strides=(2, 2))(h1_)
h1_ = Dropout(.5)(h1_)

##
h2_ = Conv2D(32, (5, 5), padding='same',
                        input_shape=input_shape)(h1_)
h2_ = PReLU()(h2_)
h2_ = BatchNormalization()(h2_)
h2_ = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h2_)
h2_ = Dropout(.5)(h2_)

##
h3_ = Conv2D(32, (3, 3), padding='same',
                        input_shape=input_shape)(h2_)
h3_ = PReLU()(h3_)
h3_ = BatchNormalization()(h3_)
h3_ = AveragePooling2D(pool_size=(3, 3),strides=(2, 2), padding='same')(h3_)
h3_ = Dropout(.5)(h3_)


h3_ = Flatten()(h3_)
h4_ = Dense(512)(h3_)
h4_ = PReLU()(h4_)
h5_ = Dropout(0.5)(h4_)
h5_ = BatchNormalization()(h5_)
h_class_ = Dense(50, activation = 'relu')(h5_)
prediction_multi = Dense(7, activation='softmax',name= 'output_multi')(h_class_)
label_multi = Input((7,))

In [12]:
def custom_loss(prediction_multi, prediction_binary, label_multi, label_binary, alpha):
    def loss(y_true, y_pred):
        return (1-alpha) * K.pow(categorical_crossentropy(prediction_multi, label_multi), 2) \
                        + alpha * K.pow(binary_crossentropy(prediction_binary, label_binary), 0.5)
    return loss

In [13]:
import tensorflow as tf
from keras.optimizers import Adam
from keras import metrics
from keras.callbacks import EarlyStopping,TensorBoard,History, ReduceLROnPlateau
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.optimizers import Adamax, Adadelta

hist_vanilla = History()

#rdLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 0, 
                              #mode = 'auto', cooldown = 0, min_lr = 0)

model_callbacks = [hist_vanilla]


losses = {
    'output_multi':'categorical_crossentropy',
    'output_binary': 'binary_crossentropy'
}

lossWeights = {'output_multi':1.0, 'output_binary': 1.0}


#model_joint = Model(inputs=[x_multi, x_binary, label_multi, label_binary], outputs = [prediction_multi, prediction_binary])
#model.add_loss(emo_loss_fixed(y_true, y_pred, arousal))

#loss_joint = K.mean(K.sum(categorical_crossentropy(prediction_multi, label_multi))+
                          #K.sum(binary_crossentropy(prediction_binary, label_binary)))
model_joint = Model(inputs = [x_multi,x_binary], outputs = [prediction_multi, prediction_binary])
model_joint.compile(loss= custom_loss(prediction_multi, prediction_binary, label_multi, label_binary, 0.50), 
                    optimizer = Adadelta() , metrics = ['accuracy'], target_tensors = [label_multi, label_binary])
#model.compile(loss = None, optimizer='adam', metrics=[acc])

In [14]:
y_train_soft = np.load('proactive_label_fer2013_v6_divide12_bound2.5*7.npy')

In [29]:
num_epochs = 50 # 
batch_size = 100

model_joint.fit([X_train, X_train_binary],[y_train_soft, y_train_binary], 
          batch_size = batch_size,
          epochs = num_epochs,
          verbose= 2,
          callbacks=model_callbacks,
          shuffle=True,
          ) ## works but overfit 

Epoch 1/50
 - 334s - loss: 8.6980 - output_multi_loss: 4.3490 - output_binary_loss: 4.3490 - output_multi_acc: 0.6525 - output_binary_acc: 0.9999
Epoch 2/50
 - 318s - loss: 8.7414 - output_multi_loss: 4.3707 - output_binary_loss: 4.3707 - output_multi_acc: 0.6544 - output_binary_acc: 0.9999
Epoch 3/50
 - 318s - loss: 8.7530 - output_multi_loss: 4.3765 - output_binary_loss: 4.3765 - output_multi_acc: 0.6537 - output_binary_acc: 1.0000
Epoch 4/50
 - 317s - loss: 8.7100 - output_multi_loss: 4.3550 - output_binary_loss: 4.3550 - output_multi_acc: 0.6549 - output_binary_acc: 1.0000
Epoch 5/50
 - 317s - loss: 8.7943 - output_multi_loss: 4.3972 - output_binary_loss: 4.3972 - output_multi_acc: 0.6522 - output_binary_acc: 1.0000
Epoch 6/50
 - 314s - loss: 8.6698 - output_multi_loss: 4.3349 - output_binary_loss: 4.3349 - output_multi_acc: 0.6565 - output_binary_acc: 1.0000
Epoch 7/50
 - 318s - loss: 8.6707 - output_multi_loss: 4.3354 - output_binary_loss: 4.3354 - output_multi_acc: 0.6583 - outp

In [33]:
####
import pandas as pd

#vanilla_cnn_fer2013 = hist_vanilla.history

#pd.DataFrame(vanilla_cnn_fer2013).to_csv('emo-focal_dualNet_softlabel_last50ep.csv')

In [30]:
X_train_binary_ = X_train_binary[:3589]
y_train_binary_ = y_train_binary[:3589]

In [31]:
## Testing the model

score = model_joint.evaluate([X_test, X_train_binary_], [y_test, y_train_binary_])

3589/3589 [==============================] - 19s 5ms/step

In [32]:
score


[87.46136066461214,
 43.73068033230607,
 43.73068033230607,
 0.6168849261715805,
 1.0]